In [1]:
%run preamble_scripts.py

# Simulation study: the well specified case 500 observations

In [2]:
expo = 500
# Setting the true model
f1, f2 = bs.loss_model("Gamma", ["r1", "m1"]), bs.loss_model("Lomax", ["α2", "σ2"])
f_true = bs.spliced_loss_model(f1, f2, "continuous")
parms_true = np.array([1/2, 1, 2.5, 3, 1.5])
f_true.set_ppf(), f_true.set_pdf(),f_true.set_cdf()

# We set the priority to the 90% quantile and the limit to the 0.99% quantile
P, L = f_true.ppf(parms_true, 0.9), f_true.ppf(parms_true, 0.99)
premiums = f_true.PP(parms_true), f_true.XOLP(parms_true, P, L)

PnLs = np.array(f_true.PnL(parms_true, P, L, expo, premiums, safety_loadings = [0.05, 0.05], n_sim = 10))

We are interested in the estimations of the extreme quantile of the claim size distribution (of order 0.95, 0.99, 0.995) and the quantile of the aggregate losses over one year with a XOL reinsurance agreement

In [3]:
true_VaRs = [f_true.ppf(parms_true, prob) for prob in [0.95, 0.99, 0.995]]
true_cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
true_cap

array([-63.74505557, -61.67416377, -45.10702938])

In [4]:
# Model for the bulk distribution
body_model_names = ["Exp", "Gamma", "Weibull", "Lognormal", "Inverse-Weibull", "Inverse-Gamma", "Inverse-Gaussian", "Lomax", "Log-Logistic", "Burr"]
body_model_param_names = [ ["λ1"], ["r1", "m1"], ["k1", "β1"],
                          ["μ1", "σ1"], ["k1", "β1"], ["r1", "m1"], ["μ1", "λ1"], ["α1", "σ1"], ["β1", "σ1"], ["α1", "β1", "σ1"] ]

# Prior distributions over the parameters of the bulk distribution
body_model_priors= [ 
    [bs.prior_model('gamma',body_model_param_names[0][0], 1, 1)], 
     [bs.prior_model('gamma',body_model_param_names[1][0], 1, 1), bs.prior_model('gamma',body_model_param_names[1][1], 1, 1)],
    [bs.prior_model('gamma',body_model_param_names[2][0], 1, 1), bs.prior_model('gamma',body_model_param_names[2][1], 1, 1)],
    [bs.prior_model('normal',body_model_param_names[3][0], 0, 0.5), bs.prior_model('gamma',body_model_param_names[3][1], 1, 1)],
     [bs.prior_model('gamma',body_model_param_names[4][0], 1, 1), bs.prior_model('gamma',body_model_param_names[4][1], 1, 1)], 
    [bs.prior_model('gamma',body_model_param_names[5][0], 1, 1), bs.prior_model('gamma',body_model_param_names[5][1], 1, 1)], 
    [bs.prior_model('gamma',body_model_param_names[6][0], 1, 1), bs.prior_model('gamma',body_model_param_names[6][1], 1, 1)], 
    [bs.prior_model('gamma',body_model_param_names[7][0], 1, 1), bs.prior_model('gamma',body_model_param_names[7][1], 1, 1)], 
    [bs.prior_model('gamma',body_model_param_names[8][0], 1, 1), bs.prior_model('gamma',body_model_param_names[8][1], 1, 1)],
    [bs.prior_model('gamma',body_model_param_names[9][0], 1, 1), bs.prior_model('gamma',body_model_param_names[9][1], 1, 1), 
     bs.prior_model('gamma',body_model_param_names[9][2], 1, 1)]
]

# Model for the tail of the distribution
tail_model_names = ["Weibull", "Lognormal", "Log-Logistic", "Lomax", "Burr", "Pareto-Tail", "GPD-Tail", "Inverse-Gamma", "Inverse-Weibull", "Exp", "Gamma"]

tail_model_param_names = [["k2", "β2"], ["μ2", "σ2"], ["β2", "σ2"], ["α2", "σ2"], ["α2", "β2", "σ2"], ["α2"], ["ξ2","σ2"], ["r2", "m2"], ["k2", "β2"], ["λ2"], ["r2", "m2"]]

# Prior distributions over the parameters of the bulk distribution
tail_model_priors= [
                [bs.prior_model('gamma',tail_model_param_names[0][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[0][1], 1, 1)],
                [bs.prior_model('normal',tail_model_param_names[1][0], 0, 0.5), bs.prior_model('gamma',tail_model_param_names[1][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[2][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[2][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[3][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[3][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[4][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[4][1], 1, 1), bs.prior_model('gamma',tail_model_param_names[4][2], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[5][0], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[6][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[6][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[7][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[7][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[8][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[8][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[9][0], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[10][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[10][1], 1, 1)]
]

γ_prior = bs.prior_model('gamma',"γ", 1, 1)

#Splicing model type
splicing_types = ["continuous"]

# Setting the models
fs, f_names, prior_spliced_model = [], [], []
for i in range(len(body_model_names)):
    for j in range(len(tail_model_names)):
        for splicing_type in splicing_types:
            f1, f2 =  bs.loss_model(body_model_names[i], body_model_param_names[i]), bs.loss_model(tail_model_names[j], tail_model_param_names[j])
            fs.append(bs.spliced_loss_model(f1 , f2, splicing_type))
            f_names.append(body_model_names[i] +"_"+ tail_model_names[j]+"_"+splicing_type)
            if splicing_type == "disjoint": 
                prior_spliced_model.append(bs.independent_priors(body_model_priors[i] + tail_model_priors[j] + [γ_prior, p_prior]))
            else:
                prior_spliced_model.append(bs.independent_priors(body_model_priors[i] + tail_model_priors[j] + [γ_prior]))  
for f in fs:
    f.set_ppf(), f.set_cdf(), f.set_pdf() 
f_spliced_dic = dict(zip(f_names, fs))
prior_dic = dict(zip(f_names, prior_spliced_model))
len(fs)

110

In [5]:
nobs, n_sim = expo, 100
Xs = [f_true.sample(parms_true, nobs) for k in range(n_sim)]
popSize, ρ, c, n_step_max, err, paralell, n_proc, verbose = 5000, 1/2, 0.99, 25, 1e-6, False, 4, False
dfs = []
for k in range(n_sim):
    print("Simulation #"+str(k))
    def fit_spliced_models(i):
        trace, log_marg, DIC, WAIC = bs.smc(Xs[k], fs[i], popSize, prior_spliced_model[i], ρ, c,n_step_max, err, paralell, 4, verbose)
        VaRs = [fs[i].ppf(trace.mean().values, prob) for prob in [0.95, 0.99, 0.995]]
#         premiums = fs[i].PP(trace.mean().values), fs[i].XOLP(trace.mean().values, P, L)
#         PnLs = np.array(fs[i].PnL(trace.mean().values, P, L, expo, premiums, safety_loadings = [0.05, 0.05], n_sim = int(1e5)))
#         caps = np.quantile(PnLs, [0.005, 0.01, 0.05])
        Wass_dist = bs.compute_Wasserstein(Xs[k], fs[i], trace.mean().values, 1)
        return(np.array([k, f_names[i], nobs, trace["γ"].mean(), log_marg, Wass_dist] + VaRs))
    %time res = Parallel(n_jobs= 40)(delayed(fit_spliced_models)(i) for i in range(len(fs)))
    df = pd.DataFrame(res, columns = ["sim", "model_name", "nobs", "γ_map", "log_marg", "Wass_dist", "q95", "q99", "q995"])
    df[df.columns[2:]] = df[df.columns[2:]].astype(float)

    df["posterior_probability"] = np.exp(df["log_marg"] - np.max(df["log_marg"])) / np.sum(np.exp(df["log_marg"] - np.max(df["log_marg"]))) 
    dfs.append(df)


Simulation #0
CPU times: user 3min 51s, sys: 1.17 s, total: 3min 52s
Wall time: 5min 4s
Simulation #1
CPU times: user 3min 3s, sys: 488 ms, total: 3min 3s
Wall time: 4min 40s
Simulation #2
CPU times: user 3min 24s, sys: 524 ms, total: 3min 24s
Wall time: 5min 2s
Simulation #3
CPU times: user 3min 54s, sys: 292 ms, total: 3min 54s
Wall time: 5min 32s
Simulation #4
CPU times: user 3min 44s, sys: 564 ms, total: 3min 44s
Wall time: 5min 29s
Simulation #5
CPU times: user 3min 31s, sys: 396 ms, total: 3min 32s
Wall time: 4min 32s
Simulation #6
CPU times: user 3min 35s, sys: 600 ms, total: 3min 35s
Wall time: 5min 5s
Simulation #7
CPU times: user 3min 41s, sys: 480 ms, total: 3min 41s
Wall time: 5min 4s
Simulation #8
CPU times: user 3min 54s, sys: 356 ms, total: 3min 54s
Wall time: 5min 19s
Simulation #9
CPU times: user 3min 52s, sys: 328 ms, total: 3min 53s
Wall time: 5min 22s
Simulation #10
CPU times: user 3min 50s, sys: 392 ms, total: 3min 50s
Wall time: 5min 19s
Simulation #11
CPU times: 

/home/pgoffard/miniconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 3min 54s, sys: 1.02 s, total: 3min 55s
Wall time: 5min 33s
Simulation #16
CPU times: user 3min 52s, sys: 901 ms, total: 3min 53s
Wall time: 5min 40s
Simulation #17
CPU times: user 3min 53s, sys: 688 ms, total: 3min 53s
Wall time: 5min 24s
Simulation #18
CPU times: user 3min 33s, sys: 352 ms, total: 3min 33s
Wall time: 4min 28s
Simulation #19
CPU times: user 3min 26s, sys: 340 ms, total: 3min 26s
Wall time: 5min 7s
Simulation #20
CPU times: user 3min 41s, sys: 384 ms, total: 3min 41s
Wall time: 5min 31s
Simulation #21
CPU times: user 3min 53s, sys: 604 ms, total: 3min 53s
Wall time: 5min 31s
Simulation #22
CPU times: user 3min 48s, sys: 340 ms, total: 3min 48s
Wall time: 5min 27s
Simulation #23
CPU times: user 3min 5s, sys: 320 ms, total: 3min 5s
Wall time: 4min 35s
Simulation #24
CPU times: user 3min 36s, sys: 324 ms, total: 3min 36s
Wall time: 5min 1s
Simulation #25
CPU times: user 3min 49s, sys: 404 ms, total: 3min 50s
Wall time: 5min 20s
Simulation #26
CPU times: use

ZeroDivisionError: division by zero

Simulation #30
CPU times: user 3min 48s, sys: 945 ms, total: 3min 49s
Wall time: 5min 26s
Simulation #31
CPU times: user 3min 45s, sys: 268 ms, total: 3min 45s
Wall time: 5min 20s
Simulation #32
CPU times: user 3min 48s, sys: 356 ms, total: 3min 49s
Wall time: 5min 9s
Simulation #33
CPU times: user 2min 24s, sys: 272 ms, total: 2min 25s
Wall time: 3min 39s
Simulation #34
CPU times: user 3min 47s, sys: 568 ms, total: 3min 47s
Wall time: 5min 14s
Simulation #35
CPU times: user 3min 58s, sys: 460 ms, total: 3min 58s
Wall time: 5min 37s
Simulation #36
CPU times: user 3min 46s, sys: 340 ms, total: 3min 47s
Wall time: 5min 18s
Simulation #37
CPU times: user 3min 47s, sys: 304 ms, total: 3min 48s
Wall time: 5min 30s
Simulation #38
CPU times: user 3min 6s, sys: 352 ms, total: 3min 7s
Wall time: 4min 37s
Simulation #39
CPU times: user 3min 39s, sys: 388 ms, total: 3min 40s
Wall time: 5min 4s
Simulation #40
CPU times: user 4min 2s, sys: 388 ms, total: 4min 2s
Wall time: 5min 35s
Simulation #41
C

In [6]:
pd.concat(dfs).to_csv("../../Data/Simulations/simu_well_spec_"+str(expo)+".csv", sep=',')
with open('../../Data/Simulations/sim_data_'+str(expo)+'.obj', 'wb') as fp:
    pickle.dump(Xs, fp)

In [7]:
# df_final = pd.concat(dfs)
# df_final[["model_name","posterior_probability"]][df_final.model_name == "Gamma_Lomax_continuous"].boxplot()
# df_final

In [8]:
# for k in range(n_sim):
#     best_models= df_final[df_final.sim == str(k)].sort_values(by='log_marg', ascending=False).iloc[:10]
#     print(best_models)